## Demo_faiss_ORB

Reference:
- [const FLANN_INDEX_HIERARCHICAL](https://docs.opencv.org/3.4/dc/d8c/namespacecvflann.html)

## Run name

In [1]:
import time

project_name = 'Google_LandMark_Rec'
step_name = 'Demo_faiss_ORB'
time_str = time.strftime("%Y%m%d_%H%M%S", time.localtime())
run_name = project_name + '_' + step_name + '_' + time_str
print('run_name: ' + run_name)
t0 = time.time()

run_name: Google_LandMark_Rec_Demo_faiss_ORB_20180528_135813


## Important params

In [2]:
feature_run_name = 'Google_LandMark_Rec_FeatureExtraction_ORB_Parallel_20180519_141925'
# feature_run_name = 'Google_LandMark_Rec_FeatureExtraction_ORB_Parallel_20180520_125411'
print(feature_run_name)

Google_LandMark_Rec_FeatureExtraction_ORB_Parallel_20180519_141925


## Import PKGs

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline
from IPython.display import display

import os
import sys
import gc
import math
import shutil
import zipfile
import pickle
import h5py
import cv2
from PIL import Image

from tqdm import tqdm
import multiprocessing

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score

cpu_amount = multiprocessing.cpu_count()
print('cpu_amount: ', cpu_amount)

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


cpu_amount:  16


## Project folders

In [4]:
cwd = os.getcwd()
feature_folder = os.path.join(cwd, 'feature')
input_folder = os.path.join(cwd, 'input')
output_folder = os.path.join(cwd, 'output')
model_folder = os.path.join(cwd, 'model')

org_train_folder = os.path.join(input_folder, 'org_train')
org_test_folder = os.path.join(input_folder, 'org_test')
train_folder = os.path.join(input_folder, 'data_train')
val_folder = os.path.join(input_folder, 'data_val')
test_folder = os.path.join(input_folder, 'data_test')
test_sub_folder = os.path.join(test_folder, 'test')

train_csv_file = os.path.join(input_folder, 'train.csv')
test_csv_file = os.path.join(input_folder, 'test.csv')
sample_submission_folder = os.path.join(input_folder, 'sample_submission.csv')

## Preview csv

In [5]:
train_csv = pd.read_csv(train_csv_file)
print('train_csv.shape is {0}.'.format(train_csv.shape))
display(train_csv.head(2))

test_csv = pd.read_csv(test_csv_file)
print('test_csv.shape is {0}.'.format(test_csv.shape))
display(test_csv.head(2))

train_csv.shape is (1225029, 3).


,id,url,landmark_id
0,cacf8152e2d2ae60,http://static.panoramio.com/photos/original/70...,4676
1,0a58358a2afd3e4e,http://lh6.ggpht.com/-igpT6wu0mIA/ROV8HnUuABI/...,6651


test_csv.shape is (117703, 2).


,id,url
0,000088da12d664db,https://lh3.googleusercontent.com/-k45wfamuhT8...
1,0001623c6d808702,https://lh3.googleusercontent.com/-OQ0ywv8KVIA...


In [6]:
train_id = train_csv['id']
train_landmark_id = train_csv['landmark_id']
print('len(train_landmark_id) = \t%s' % len(list(set(train_landmark_id))))

id_2_landmark_id_dict = dict(zip(train_id, train_landmark_id))
print('len(id_2_landmark_id_dict) = \t%d' % len(id_2_landmark_id_dict))

index = 0
print('id: %s, \tlandmark_id:%s' % (train_id[index], id_2_landmark_id_dict[train_id[index]]))
index = 1
print('id: %s, \tlandmark_id:%s' % (train_id[index], id_2_landmark_id_dict[train_id[index]]))

len(train_landmark_id) = 	14951
len(id_2_landmark_id_dict) = 	1225029
id: cacf8152e2d2ae60, 	landmark_id:4676
id: 0a58358a2afd3e4e, 	landmark_id:6651


## Load feature

In [7]:
%%time
def dump_pickle_feature_batch(run_name, dataset_name, batch_num, image_features):
    run_name_folder = os.path.join(feature_folder, run_name)
    if not os.path.exists(run_name_folder):
        os.mkdir(run_name_folder)
    image_features_file = os.path.join(run_name_folder, 'feature_%s_%s_b%s.pickle' % (run_name, dataset_name, batch_num))
    print('Dump: ', image_features_file, end='  ')
    print(len(image_features.keys()))
    pickle.dump(image_features, open(image_features_file, "wb"), True)

def load_pickle_feature_batch(run_name, dataset_name, batch_num):
    run_name_folder = os.path.join(feature_folder, run_name)
    image_features_file = os.path.join(run_name_folder, 'feature_%s_%s_b%s.pickle' % (run_name, dataset_name, batch_num))
    image_features = pickle.load(open(image_features_file, "rb"))
    print('Load: ', image_features_file, end='  ')
    print(len(image_features.keys()))
    return image_features

# dump_pickle_feature_batch(run_name, image_features)
image_features = load_pickle_feature_batch(feature_run_name, 'train', 0)

Load:  /data1/kaggle/landmark-recognition-challenge/feature/Google_LandMark_Rec_FeatureExtraction_ORB_Parallel_20180519_141925/feature_Google_LandMark_Rec_FeatureExtraction_ORB_Parallel_20180519_141925_train_b0.pickle  19991
CPU times: user 2.11 s, sys: 424 ms, total: 2.53 s
Wall time: 2.53 s


In [8]:
print('*'*80)
print('len_image_features=', len(image_features.keys()))
for i, image_id in enumerate(list(image_features.keys())[:3]):
    print('image_id: %s, \tlandmark_id:%s,\t feature_shape: ' % (image_id, id_2_landmark_id_dict[image_id]), image_features[image_id].shape)


********************************************************************************
len_image_features= 19991
image_id: 69b846bd58c3f09a, 	landmark_id:6051,	 feature_shape:  (500, 32)
image_id: 19a1de4f08cd0305, 	landmark_id:9179,	 feature_shape:  (500, 32)
image_id: 4ee821754ef5fd83, 	landmark_id:11301,	 feature_shape:  (500, 32)


In [9]:
def load_pickle_file_batch(image_features_file):
    image_features = pickle.load(open(image_features_file, "rb"))
    print('Load: ', image_features_file, end='  ')
    print(len(image_features.keys()))
    return image_features

In [10]:
def load_pickle_feature_all(feature_run_name, dataset_name):
    file_names = os.listdir(os.path.join(feature_folder, feature_run_name))
#     file_names = list(filter(lambda x: dataset_name in x, file_names))[:3]
    file_names = list(filter(lambda x: dataset_name in x, file_names))
    
    all_image_fatures = {}
    for file_name in file_names:
        image_features_file = os.path.join(feature_folder, feature_run_name, file_name)
        image_features = load_pickle_file_batch(image_features_file)
        all_image_fatures.update(image_features)
    return all_image_fatures

In [11]:
def load_pickle_feature_all_parallel(feature_run_name, dataset_name):
    file_names = os.listdir(os.path.join(feature_folder, feature_run_name))
#     file_names = list(filter(lambda x: dataset_name in x, file_names))[:3]
    file_names = list(filter(lambda x: dataset_name in x, file_names))
    file_names.sort()
    file_names = file_names
    all_image_fatures = {}
#     for file_name in file_names:
#         image_features_file = os.path.join(feature_folder, feature_run_name, file_name)
#         image_features = load_pickle_file_batch(image_features_file)
#         all_image_fatures.update(image_features)
#     return all_image_fatures

    image_features_files = [os.path.join(feature_folder, feature_run_name, file_name) for file_name in file_names]
    pool = multiprocessing.Pool(processes=cpu_amount)
    for image_features in tqdm(pool.imap(load_pickle_file_batch, image_features_files), total=len(image_features_files)):
        all_image_fatures.update(image_features)
    pool.close()
    pool.terminate()
    return all_image_fatures

In [12]:
%%time
all_image_fatures_train = load_pickle_feature_all_parallel(feature_run_name, 'train')

  0%|          | 0/61 [00:00<?, ?it/s]

Load:  /data1/kaggle/landmark-recognition-challenge/feature/Google_LandMark_Rec_FeatureExtraction_ORB_Parallel_20180519_141925/feature_Google_LandMark_Rec_FeatureExtraction_ORB_Parallel_20180519_141925_train_b11.pickle  19988
Load:  /data1/kaggle/landmark-recognition-challenge/feature/Google_LandMark_Rec_FeatureExtraction_ORB_Parallel_20180519_141925/feature_Google_LandMark_Rec_FeatureExtraction_ORB_Parallel_20180519_141925_train_b13.pickle  19993
Load:  /data1/kaggle/landmark-recognition-challenge/feature/Google_LandMark_Rec_FeatureExtraction_ORB_Parallel_20180519_141925/feature_Google_LandMark_Rec_FeatureExtraction_ORB_Parallel_20180519_141925_train_b10.pickle  19991
Load:  /data1/kaggle/landmark-recognition-challenge/feature/Google_LandMark_Rec_FeatureExtraction_ORB_Parallel_20180519_141925/feature_Google_LandMark_Rec_FeatureExtraction_ORB_Parallel_20180519_141925_train_b0.pickle  19991
Load:  /data1/kaggle/landmark-recognition-challenge/feature/Google_LandMark_Rec_FeatureExtraction

  2%|▏         | 1/61 [00:10<10:35, 10.60s/it]

Load:  /data1/kaggle/landmark-recognition-challenge/feature/Google_LandMark_Rec_FeatureExtraction_ORB_Parallel_20180519_141925/feature_Google_LandMark_Rec_FeatureExtraction_ORB_Parallel_20180519_141925_train_b28.pickle  19990


 13%|█▎        | 8/61 [00:11<01:14,  1.41s/it]

...

Load:  /data1/kaggle/landmark-recognition-challenge/feature/Google_LandMark_Rec_FeatureExtraction_ORB_Parallel_20180519_141925/feature_Google_LandMark_Rec_FeatureExtraction_ORB_Parallel_20180519_141925_train_b57.pickle  19995


 67%|██████▋   | 41/61 [00:34<00:16,  1.20it/s]

Load:  /data1/kaggle/landmark-recognition-challenge/feature/Google_LandMark_Rec_FeatureExtraction_ORB_Parallel_20180519_141925/feature_Google_LandMark_Rec_FeatureExtraction_ORB_Parallel_20180519_141925_train_b58.pickle  19990
Load:  /data1/kaggle/landmark-recognition-challenge/feature/Google_LandMark_Rec_FeatureExtraction_ORB_Parallel_20180519_141925/feature_Google_LandMark_Rec_FeatureExtraction_ORB_Parallel_20180519_141925_train_b59.pickle  19994
Load:  /data1/kaggle/landmark-recognition-challenge/feature/Google_LandMark_Rec_FeatureExtraction_ORB_Parallel_20180519_141925/feature_Google_LandMark_Rec_FeatureExtraction_ORB_Parallel_20180519_141925_train_b6.pickle  19996
Load:  /data1/kaggle/landmark-recognition-challenge/feature/Google_LandMark_Rec_FeatureExtraction_ORB_Parallel_20180519_141925/feature_Google_LandMark_Rec_FeatureExtraction_ORB_Parallel_20180519_141925_train_b60.pickle  17676
Load:  /data1/kaggle/landmark-recognition-challenge/feature/Google_LandMark_Rec_FeatureExtraction

100%|██████████| 61/61 [00:47<00:00,  1.27it/s]


CPU times: user 14.2 s, sys: 21.7 s, total: 35.9 s
Wall time: 49 s


In [13]:
print('*'*80)
print('len_image_features=', len(all_image_fatures_train.keys()))
for i, image_id in enumerate(list(all_image_fatures_train.keys())[:3]):
    print('image_id: %s, \tlandmark_id:%s,\t feature_shape: ' % (image_id, id_2_landmark_id_dict[image_id]), all_image_fatures_train[image_id].shape)


********************************************************************************
len_image_features= 1217193
image_id: 69b846bd58c3f09a, 	landmark_id:6051,	 feature_shape:  (500, 32)
image_id: 19a1de4f08cd0305, 	landmark_id:9179,	 feature_shape:  (500, 32)
image_id: 4ee821754ef5fd83, 	landmark_id:11301,	 feature_shape:  (500, 32)


In [14]:
key_train, key_val = train_test_split(list(all_image_fatures_train.keys()), test_size=0.95, random_state=2017)
print('len(key_train)=', len(key_train))
print('len(key_val)=', len(key_val))

len(key_train)= 60859
len(key_val)= 1156334


## faiss Match

In [15]:
import faiss
dim = 32
faiss_indx = faiss.IndexFlatL2(dim)
print(faiss_indx.is_trained)

True


In [16]:
%%time

labels = []
count = 0
for image_id in tqdm(key_train, total=len(key_train)):
    feature = all_image_fatures_train[image_id].astype('float32')
    landmark_id = id_2_landmark_id_dict[image_id]
#     print('image_id: %s, \tlandmark_id:%s,\t feature_shape: ' % (image_id, landmark_id), feature.shape)
#     print(feature[:100].shape)
    faiss_indx.add(feature)
    labels.append(np.ones(feature.shape[0]) * landmark_id)
#     del all_image_fatures_train[image_id]

    count += 1
    if count % 100000 == 0:
#         print(count)
        gc.collect()

100%|██████████| 60859/60859 [00:04<00:00, 14551.35it/s]

CPU times: user 2.67 s, sys: 1.54 s, total: 4.21 s
Wall time: 4.18 s


In [17]:
labels = np.concatenate(labels, axis=0)
print(labels.shape)
print(faiss_indx.ntotal)

(30014128,)
30014128


In [18]:
%%time
k = 1
min_des = 5
rows = 100
count = 0
for key in key_val[:rows]:
    feats = all_image_fatures_train[key].astype('float32')
    D, I = faiss_indx.search(feats, k)
#     print(D.shape)
#     print(I.shape)
    indx = I.reshape(-1)
#     des = np.array(list(map(lambda n: print(n, end=' ') if n <= minlength else n, des)))
#     print(des.shape, des)
#     indx = I.reshape(-1)
    lable = [labels[i] for i in indx]
    count_indx = np.bincount(lable)
    label = np.argsort(count_indx)
    label = label[::-1][:3]
    print(label, count_indx[label], end=' ')
#     count_des = np.array(list(map(lambda n: n if n <= minlength else 0, des)))
#     print(count_des.shape, count_des)
#     y = np.bincount(y)
#     print(y)
    print(key, id_2_landmark_id_dict[key], end=' ')
    if id_2_landmark_id_dict[key] in label:
        print('+')
        count += 1
    else:
        print('-')

[9633 6051 5554] [24 21  8] 1cb7cab636743701 4118 -
[9633 6051 9779] [26 19  6] c6e30b6ebf7e5813 1039 -
[6051 9633 6599] [17 15 13] e0a41aecee2946a8 1685 -
[9633 6051 2228] [20 16 10] bd65da3fad734987 2228 +
[6051 9633 6599] [17 16 12] b9da46d93e488704 6577 -
[9633 6051 6599] [20 18 10] 5aebac37cd82703b 8988 -
[6051 9633 6599] [24 22  9] 855a7dddb6c262e5 13060 -
[6051 9633 6599] [22 17 12] bdd71ccf855fc035 243 -
[9633 6051 2061] [59 21  8] 0dda3e0def9b87fb 9633 +
[6599 6051 9633] [15 15 14] 1e83b143d7c64d37 4936 -
[6051 9633 6599] [27 22 14] 16631bb3f7046c06 3420 -
[6051 9633 5554] [32 13 11] 312d9b3ed4e833e7 6696 -
[9633 6051 6599] [23 16 13] 26d381d08c369f70 11812 -
[6051 9633 6599] [23 18 12] cf05e576dcca6da7 503 -
[9633 6051 6599] [17 15  8] 232dd63d3dc267d1 6072 -
[6051 9633 9779] [19 17 15] 8564836c9efca669 6051 +
[6051 9633 6599] [20 16 12] 08b6077ecc4890fc 3495 -
[6051 9633 2061] [21 15  9] d06f85088c8ccebf 165 -
[9633 6051 9779] [31 21  9] 780afb0d334562c7 9039 -
[6051 9633 34

In [20]:
acc = count*1.0 / rows
print('acc=%.2f' % acc)

acc=0.20
